In [1]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch

import numpy as np
import cv2

import os  
from pathlib import Path
import sys

import matplotlib.pyplot as plt

from einops import rearrange

import glob
from sklearn.model_selection import train_test_split

In [2]:
path = Path(os.getcwd())
parent = path.parent
parent = str(parent)
path = str(path)
print(path)
sys.path.insert(1, path)
sys.path.insert(1, parent)

/Users/nelsonfarrell/Documents/Northeastern/7180/projects/spectral_ratio


In [3]:
from dataloader.dataset_generator_class import ImageDatasetGenerator
from models.CvT_model import CvT
from angular_dist_loss.angular_dist_loss import CosineDistanceMetric

In [4]:
import torch
import torch.nn.functional as F

# Create dummy image data with 3 color channels (RGB) and spatial dimensions 10x10
height, width, channels = 10, 10, 3
image1 = torch.randn(1, 3)  # Batch size = 1
image2 = torch.randn(1, 3)  # Batch size = 1

import torch

# Define two specific tensors
actual = torch.tensor([[0.0, 0.0, 0.0], [0.5, 0.5, 0.0]])  # Batch size = 2, each vector has 3 elements
predicted = torch.tensor([[0.5, 0.5, 0.0], [-0.5, -0.5, 0.0]])  # Batch size = 2, each vector has 3 elements

print("Actual Tensors:\n", actual)
print("Predicted Tensors:\n", predicted)


# Compute cosine similarity along the channel dimension (dim=1)
cosine_similarity_map = F.cosine_similarity(actual, predicted, dim = 1)  # Shape: (1, height, width)

# Inspect the result
print("Cosine Similarity Map Shape:", cosine_similarity_map.shape)
print("Cosine Similarity Map (Sample):", cosine_similarity_map)


Actual Tensors:
 tensor([[0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000]])
Predicted Tensors:
 tensor([[ 0.5000,  0.5000,  0.0000],
        [-0.5000, -0.5000,  0.0000]])
Cosine Similarity Map Shape: torch.Size([2])
Cosine Similarity Map (Sample): tensor([ 0.0000, -1.0000])


In [5]:
# Transforms -- We can add augmentations here
transform_images = transforms.Compose([ 
                        transforms.ToTensor()       
                        #transforms.Normalize(mean = mean, std = std)
                                    ])

transform_guidance = transforms.Compose([ 
                        transforms.ToTensor()
                                    ])

In [6]:
# Folder paths
image_folder = "/Users/nelsonfarrell/Documents/Northeastern/7180/projects/spectral_ratio/training_data/training_images_cropped"
isd_map_folder = "/Users/nelsonfarrell/Documents/Northeastern/7180/projects/spectral_ratio/training_data/training_isds_cropped"

# Create dataset
dataset = ImageDatasetGenerator(image_folder, 
                                isd_map_folder, 
                                split = None, 
                                val_size = 0.2, 
                                random_seed = 42, 
                                transform_images = transform_images, 
                                transform_guidance = transform_guidance)

# Create dataset
train_dataset = ImageDatasetGenerator(image_folder, 
                                      isd_map_folder, 
                                      split = "train", 
                                      val_size = 0.2, 
                                      random_seed = 42, 
                                      transform_images = transform_images, 
                                      transform_guidance = transform_guidance)

# Create dataset
val_dataset = ImageDatasetGenerator(image_folder, 
                                    isd_map_folder, 
                                    split = "val", 
                                    val_size = 0.2, 
                                    random_seed = 42, 
                                    transform_images = transform_images, 
                                    transform_guidance = transform_guidance)

# Create dataloader
# Shuffle equals false for now for testing
# drop last creates only full batches
dataloader = DataLoader(val_dataset, batch_size = 4, shuffle = False, drop_last = True)

Assertion Passed!!! We have the same number of images and ISD maps.
Assertion Passed!!! We have the same number of images and ISD maps.
Assertion Passed!!! We have the same number of images and ISD maps.


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'  
embed_size = 64
num_class = 10
model = CvT(embed_size).to(device)

Entering embedding: Patch Size: 7 -- Stride: 4 -- Embedding Dims 64
Entering embedding: Patch Size: 3 -- Stride: 2 -- Embedding Dims 192
Entering embedding: Patch Size: 3 -- Stride: 2 -- Embedding Dims 384


In [8]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Parameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 10
learning_rate = 0.001
batch_size = 4

# Define loss function and optimizer
criterion = CosineDistanceMetric(reduce = False)  
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.train()
    train_loss = 0.0

    # Training Step
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    for batch in tqdm(train_loader, desc="Training"):
        images, guidance_maps = batch
        images, guidance_maps = images.to(device), guidance_maps.to(device)

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, guidance_maps)
        print("Loss:", loss.item())

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f"Training Loss: {train_loss:.4f}")

    # Validation Step
    model.eval()
    val_loss = 0.0
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            images, guidance_maps = batch
            images, guidance_maps = images.to(device), guidance_maps.to(device)

            # Forward pass
            outputs = model(images)

            # Compute loss
            loss = criterion(outputs, guidance_maps)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Validation Loss: {val_loss:.4f}")

print("Training Complete!")


Epoch 1/10


Training:   0%|          | 0/429 [00:00<?, ?it/s]

Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
Loss: 1.184691309928894


Training:   0%|          | 1/429 [00:01<10:57,  1.54s/it]

Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
Loss: 0.889155924320221


...

Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
Loss: 1.0035547018051147


Training:   6%|▌         | 24/429 [00:27<07:27,  1.11s/it]

Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
Loss: 0.8979801535606384


Training:   6%|▌         | 25/429 [00:28<07:30,  1.11s/it]

Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])


In [7]:
import torch

input_ = torch.ones((10, 3, 74, 74))

print(input_.shape)
# torch.Size([10, 1, 74, 74])

flattened_input = input_.view(10, -1)

print(flattened_input.shape)
# torch.Size([10, 5476])



torch.Size([10, 3, 74, 74])
torch.Size([10, 16428])
